In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading


In [7]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [8]:
def scrape_links(page_number):
    url = f"https://www.krjogja.com/indeks-berita?daterange=&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h2',{"class": "latest__title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [9]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 1
Total Links: 10


In [10]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Judul Berita
            title_elem = soup.find('h1',{"class": "read__title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # # Author berita
            author_elem = soup.find('div', {"class": "read__info__author"})
            if author_elem:
                penulis = author_elem.find('a')
                author_text =penulis.text
            else:
                author_text = "Author not found"
            
            # # tanggal berita
            date_elem = soup.find('div', {"class": "read__info__date"})
            if date_elem:
                date_text = date_elem.text
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find_all('li', {"class": "breadcrumb__item"})
            if category_elements:
                category_text= category_elements[1].text
            else:
                category_text = "Category not found"
            # ## Content Berita
            body_elem = soup.find('article', {"class": "read__content clearfix"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'author': author_text,
                            'date':date_text,
                            'category': category_text,
                            'content': content_text,
                            'region':'jogja',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [11]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [12]:
df = pd.DataFrame(results)
print("Total Links:", len(results))
df.head(10)

Total Links: 10


,title,author,date,category,content,region,link
0,Herbalife Run Ajak Masyarakat Jakankan Gaya H...,Rini Suryati,"\n - Kamis,...",Olahraga\n,Krjogja.com - Jakarta - Sebanyak 5000 peserta ...,jogja,https://www.krjogja.com/olahraga/1243025487/he...
1,Ilmuwan dari China Temukan Virus Misterius di...,Tomi Sujatmiko,"\n - Kamis,...",Teknologi\n,\nKrjogja.com - Jakarta - Qingdao - Palung Mar...,jogja,https://www.krjogja.com/teknologi/1243024968/i...
2,"Terlibat Kasus Korupsi, Broker Tanah Jadi DPO...",Karyono KrJogja,"\n - Kamis,...",Semarang\n,"Krjogja.com - SEMARANG - JA, seorang broker di...",jogja,https://www.krjogja.com/semarang/1243025008/te...
3,Bareksa dan Danamon Hadirkan Direct Debit Pem...,Tomi Sujatmiko,"\n - Kamis,...",Keuangan\n,Jakarta - Krjogja.com - Platform investasi Bar...,jogja,https://www.krjogja.com/keuangan/1243024497/ba...
4,"Hasil Survei, Prabowo Subianto Unggul Hea...",Tomi Sujatmiko,"\n - Kamis,...",Nasional\n,\nJakarta - Bakal calon presiden (capres) Prab...,jogja,https://www.krjogja.com/nasional/1243024910/ha...
5,"Awas, Modus Penipuan dengan Action Button 'VI...",Tomi Sujatmiko,"\n - Kamis,...",Teknologi\n,Krjogja.com - Jakarta- Berbagai modus penipuan...,jogja,https://www.krjogja.com/teknologi/1243024951/a...
6,Parade Desa Wisata Puncak Jogja Tourism Day d...,Asrul Sani,"\n - Kamis,...",KulonProgo\n,Krjogja.com - Kulonprogo - Puncak Jogja Touris...,jogja,https://www.krjogja.com/kulonprogo/1243024988/...
7,Chadis Rooftop Bar Swiss-belboutique 'Disulap...,Tomi Sujatmiko,"\n - Kamis,...",Yogyakarta\n,Krjogja.com - YOGYA - Swiss-belboutique Yogyak...,jogja,https://www.krjogja.com/yogyakarta/1243024984/...
8,"Sampai 2035, Inggris Tunda Larangan Penjual...",Tomi Sujatmiko,"\n - Kamis,...",Otomotif\n,Krjogja.com - Jakarta - Pemerintah Inggris tel...,jogja,https://www.krjogja.com/otomotif/1243024892/sa...
9,Elkan Baggott Cs Kembali akan Bertemu Klub Pr...,Primaswolo Sudjono,"\n - Kamis,...",Olahraga\n,\nKrjogja.com - IPSWICH - Klub Elkan Baggott a...,jogja,https://www.krjogja.com/olahraga/1243025310/el...
